In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# 화해

In [3]:
# 데이터 불러오기
hwahae = pd.read_csv('통합_화해_데이터.csv')

# 데이터셋 복제
hwahae_df = hwahae.copy(deep=True)

In [4]:
# 데이터셋 기본 정보 조회
print(hwahae_df.info())
print(hwahae_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   중분류     4781 non-null   object 
 6   소분류     0 non-null      float64
 7   평점      4781 non-null   float64
 8   후기 수    4781 non-null   int64  
 9   가격      4686 non-null   float64
 10  URL     0 non-null      float64
dtypes: float64(4), int64(2), object(5)
memory usage: 411.0+ KB
None
  플랫폼    브랜드                                        제품명  중분류_순위   대분류   중분류  \
0  화해  에스네이처         아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜)       1  스킨케어  스킨토너   
1  화해    토리든  다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml)       2  스킨케어  스킨토너   
2  화해   라운드랩             1025 독도 토너 200ml + 독도 토너 500ml       3  스킨케어  스킨토너   

   소분류    평점   후기 수       

## 기본 전처리

In [ ]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 1) 후기 수 & 가격 컬럼의 쉼표 없애기
# 원본 데이터셋에 전처리 되어있음

'''
for i in range(len(hwahae_df)):
  score = hwahae_df['평점'][i]
  review = hwahae_df['후기 수'][i]
  price = hwahae_df['가격'][i]
  
  score_result = score.replace('별점 없음', '0')
  review_result = review.replace(',', '')
  price_result = price.replace(',', '')
  
  hwahae_df.loc[i, '평점'] = score_result
  hwahae_df.loc[i, '후기 수'] = review_result
  hwahae_df.loc[i, '가격'] = price_result

print(hwahae_df.loc[:10, ['평점', '후기 수', '가격']])
'''

"\nfor i in range(len(hwahae_df)):\n  score = hwahae_df['평점'][i]\n  review = hwahae_df['후기 수'][i]\n  price = hwahae_df['가격'][i]\n  \n  score_result = score.replace('별점 없음', '0')\n  review_result = review.replace(',', '')\n  price_result = price.replace(',', '')\n  \n  hwahae_df.loc[i, '평점'] = score_result\n  hwahae_df.loc[i, '후기 수'] = review_result\n  hwahae_df.loc[i, '가격'] = price_result\n\nprint(hwahae_df.loc[:10, ['평점', '후기 수', '가격']])\n"

In [ ]:
# 1. 컬럼 타입 변경 : 평점(float), 후기 수(int), 가격(int)
# 2) 컬럼 타입 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df[['평점', '후기 수', '가격']] = hwahae_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)

print(hwahae_df[['평점', '후기 수', '가격']].dtypes)
print(hwahae_df.loc[:5, ['평점', '후기 수', '가격']])
'''

"\nhwahae_df[['평점', '후기 수', '가격']] = hwahae_df[['평점', '후기 수', '가격']].apply(pd.to_numeric)\n\nprint(hwahae_df[['평점', '후기 수', '가격']].dtypes)\nprint(hwahae_df.loc[:5, ['평점', '후기 수', '가격']])\n"

In [40]:
# 2. 컬럼 이름 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)
hwahae_df.columns
'''

"\nhwahae_df.rename(columns = {'순위':'중분류_순위', '카테고리':'대분류'}, inplace=True)\nhwahae_df.columns\n"

In [ ]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

'''
hwahae_df['플랫폼'] = '화해'

a = hwahae_df.columns[-1:].to_list()

ex_a = hwahae_df.drop(columns = hwahae_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
hwahae_df = hwahae_df[reorder]

hwahae_df.columns
'''

"\nhwahae_df['플랫폼'] = '화해'\n\na = hwahae_df.columns[-1:].to_list()\n\nex_a = hwahae_df.drop(columns = hwahae_df.columns[[-1]]).columns[:].to_list()\n\nreorder = a + ex_a\nhwahae_df = hwahae_df[reorder]\n\nhwahae_df.columns\n"

## 카테고리 전처리

In [9]:
# 1. 중분류1 & 중분류2 컬럼 생성
hwahae_df['중분류1'] = ''
print(hwahae_df['중분류1'].head())

hwahae_df['중분류2'] = ''
print(hwahae_df['중분류2'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류2, dtype: object


In [10]:
# 기존 분류별 unique 값 확인
print(hwahae_df['대분류'].unique())
print(hwahae_df['중분류'].unique())
print(hwahae_df['소분류'].unique())

['스킨케어' '클렌징' '마스크_팩' '선케어' '베이스메이크업' '아이메이크업' '립메이크업']
['스킨토너' '로션_에멀젼' '에센스_앰플_세럼' '페이스오일' '크림' '아이케어' '미스트' '젤' '스킨_토너패드'
 '밤_멀티밤' '클렌징폼' '클렌징워터' '클렌징젤' '클렌징오일' '클렌징로션_크림' '클렌징비누' '클렌징티슈_패드'
 '립_아이리무버' '스크럽_필링' '스크럽_필링패드' '클렌징파우더' '클렌징밤' '시트마스크' '부분마스크_팩' '워시오프팩'
 '필오프팩' '슬리핑팩' '패치' '코팩' '부분마스크패드' '선크림_로션' '선스프레이' '선케어기타' '선스틱' '선쿠션_팩트'
 '메이크업베이스' '프라이머' 'BB_CC크림' '파운데이션' '쿠션' '파우더_팩트' '컨실러' '블러셔' '하이라이터'
 '메이크업픽서' '셰이딩' '톤업크림' '아이섀도' '아이라이너' '아이브로우' '마스카라_픽서' '속눈썹영양제' '립스틱'
 '립틴트' '립글로스' '립케어_립밤' '컬러립케어_립밤']
[nan]


In [ ]:
# 기타 컬럼 소속 예정 값의 대분류 값 확인
cond1 = hwahae_df['중분류'] == '아이케어'
print(hwahae_df.loc[cond1, '대분류'].unique()) # 스킨케어

cond2 = hwahae_df['중분류'] == '미스트'
print(hwahae_df.loc[cond2, '대분류'].unique()) # 스킨케어

cond3 = hwahae_df['중분류'] == '젤'
print(hwahae_df.loc[cond3, '대분류'].unique()) # 스킨케어

['스킨케어']
['스킨케어']
['스킨케어']


In [ ]:
# 통합 필요한 세부 카테고리 확인
cond1 = hwahae_df['대분류'] == '스킨케어'
print(hwahae_df.loc[cond1, '중분류'].unique()) 

cond2 = hwahae_df['대분류'] == '베이스메이크업'
print(hwahae_df.loc[cond2, '중분류'].unique()) 

cond3 = hwahae_df['대분류'] == '아이메이크업'
print(hwahae_df.loc[cond3, '중분류'].unique()) 

['스킨토너' '로션_에멀젼' '에센스_앰플_세럼' '페이스오일' '크림' '아이케어' '미스트' '젤' '스킨_토너패드'
 '밤_멀티밤']
['메이크업베이스' '프라이머' 'BB_CC크림' '파운데이션' '쿠션' '파우더_팩트' '컨실러' '블러셔' '하이라이터'
 '메이크업픽서' '셰이딩' '톤업크림']
['아이섀도' '아이라이너' '아이브로우' '마스카라_픽서' '속눈썹영양제']


In [21]:
# 중분류2
# 스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
# 베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제

skin_list = ['패드', '스킨', '토너']
skin = '|'.join(skin_list)
cond_skin = hwahae_df['중분류'].str.contains(skin)

found_list = ['파운데이션', '쿠션', 'BB', 'CC']
found  = '|'.join(found_list)
cond_found = hwahae_df['중분류'].str.contains(found)

fixer_list = ['파우더', '팩트', '메이크업픽서']
fixer = '|'.join(fixer_list)
cond_fixer = hwahae_df['중분류'].str.contains(fixer)

shading_list = ['블러셔', '쉐이딩', '셰이딩', '하이라이터']
shading = '|'.join(shading_list)
cond_shading = hwahae_df['중분류'].str.contains(shading)

primer_list = ['프라이머', '베이스', '톤업크림'] # 프라이머/베이스 컬럼에 (소분류)톤업크림 추가
primer = '|'.join(primer_list)
cond_primer = hwahae_df['중분류'].str.contains(primer)

for i in range(len(hwahae_df)):
  if cond_skin[i] == True:
    hwahae_df.loc[i, '중분류2'] = '토너/토너패드/스킨'
  
  elif cond_found[i] == True:
    hwahae_df.loc[i, '중분류2'] = '파운데이션/쿠션/BB/CC'

  elif cond_fixer[i] == True:
    hwahae_df.loc[i, '중분류2'] = '파우더/팩트/픽서'
  
  elif cond_shading[i] == True:
    hwahae_df.loc[i, '중분류2'] = '블러셔/쉐이딩/하이라이터'

  elif cond_primer[i] == True:
    hwahae_df.loc[i, '중분류2'] = '프라이머/베이스'
      
  else:
    if hwahae_df['중분류'][i] == '로션_에멀젼':
      hwahae_df.loc[i, '중분류2'] = '로션'

    elif hwahae_df['중분류'][i] == '에센스_앰플_세럼':
      hwahae_df.loc[i, '중분류2'] = '에센스/세럼/앰플'
    
    elif hwahae_df['중분류'][i] == '크림':
      hwahae_df.loc[i, '중분류2'] = '크림'

    elif hwahae_df['대분류'][i] == '마스크_팩':
      hwahae_df.loc[i, '중분류2'] = '마스크팩'

    elif hwahae_df['대분류'][i] == '클렌징':
      hwahae_df.loc[i, '중분류2'] = '클렌징'

    elif hwahae_df['대분류'][i] == '선케어':
      hwahae_df.loc[i, '중분류2'] = '선케어'

    elif hwahae_df['대분류'][i] == '립메이크업':
      hwahae_df.loc[i, '중분류2'] = '립메이크업'
    
    elif hwahae_df['중분류'][i] == '컨실러':
      hwahae_df.loc[i, '중분류2'] = '컨실러'
      
    elif hwahae_df['중분류'][i] == '아이라이너':
      hwahae_df.loc[i, '중분류2'] = '아이라이너'

    elif hwahae_df['중분류'][i] == '아이섀도':
      hwahae_df.loc[i, '중분류2'] = '아이섀도우'
      
    elif hwahae_df['중분류'][i] == '아이브로우':
      hwahae_df.loc[i, '중분류2'] = '아이브로우'

    elif hwahae_df['중분류'][i] == '마스카라_픽서':
      hwahae_df.loc[i, '중분류2'] = '마스카라'
    
    elif hwahae_df['중분류'][i] == '속눈썹영양제':
      hwahae_df.loc[i, '중분류2'] = '속눈썹영양제'
      
    else: # 기획세트
      hwahae_df.loc[i, '중분류2'] = '기타'

In [22]:
print(hwahae_df['중분류2'].unique())
print(hwahae_df['중분류2'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류2']]

['토너/토너패드/스킨' '로션' '에센스/세럼/앰플' '기타' '크림' '클렌징' '파우더/팩트/픽서' '마스크팩' '선케어'
 '파운데이션/쿠션/BB/CC' '프라이머/베이스' '컨실러' '블러셔/쉐이딩/하이라이터' '아이섀도우' '아이라이너' '아이브로우'
 '마스카라' '속눈썹영양제' '립메이크업']
19


,대분류,중분류,소분류,중분류2
0,스킨케어,스킨토너,NaN,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,토너/토너패드/스킨
...,...,...,...,...
4776,립메이크업,컬러립케어_립밤,NaN,립메이크업
4777,립메이크업,컬러립케어_립밤,NaN,립메이크업
4778,립메이크업,컬러립케어_립밤,NaN,립메이크업
4779,립메이크업,컬러립케어_립밤,NaN,립메이크업


In [26]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = hwahae_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = hwahae_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = hwahae_df['중분류2'].str.contains(eye)

for i in range(len(hwahae_df)):
  if cond_skincare[i] == True:
    hwahae_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    hwahae_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    hwahae_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if hwahae_df['중분류2'][i] == '클렌징':
      hwahae_df.loc[i, '중분류1'] = '클렌징'

    elif hwahae_df['중분류2'][i] == '마스크팩':
      hwahae_df.loc[i, '중분류1'] = '마스크팩'
      
    elif hwahae_df['중분류2'][i] == '기타':
      hwahae_df.loc[i, '중분류1'] = '기타'
      
    elif hwahae_df['중분류2'][i] == '선케어':
      hwahae_df.loc[i, '중분류1'] = '선케어'
    
    elif hwahae_df['중분류2'][i] == '립메이크업':
      hwahae_df.loc[i, '중분류1'] = '립메이크업'

In [27]:
print(hwahae_df['중분류1'].unique())
print(hwahae_df['중분류1'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '기타' '클렌징' '베이스메이크업' '마스크팩' '선케어' '아이메이크업' '립메이크업']
8


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
...,...,...,...,...,...
4776,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4777,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4778,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4779,립메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업


In [28]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = hwahae_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = hwahae_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = hwahae_df['중분류1'].str.contains(colorclass)

for i in range(len(hwahae_df)):
  if cond_skinclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    hwahae_df.loc[i, '대분류'] = '색조메이크업'

In [29]:
# 대분류 고유값 확인 
print(hwahae_df['대분류'].unique())
print(hwahae_df['대분류'].nunique())
hwahae_df.loc[:, ['대분류', '중분류', '소분류', '중분류1', '중분류2']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,중분류,소분류,중분류1,중분류2
0,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
1,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
2,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
3,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
4,스킨케어,스킨토너,NaN,스킨케어,토너/토너패드/스킨
...,...,...,...,...,...
4776,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4777,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4778,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업
4779,색조메이크업,컬러립케어_립밤,NaN,립메이크업,립메이크업


## 립메이크업 중분류 세분화

In [30]:
# musinsa_df
hwahae_df.head()

,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류,소분류,평점,후기 수,가격,URL,중분류1,중분류2
0,화해,에스네이처,아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜),1,스킨케어,스킨토너,NaN,4.73,17212,18900.0,NaN,스킨케어,토너/토너패드/스킨
1,화해,토리든,다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml),2,스킨케어,스킨토너,NaN,4.69,25849,19900.0,NaN,스킨케어,토너/토너패드/스킨
2,화해,라운드랩,1025 독도 토너 200ml + 독도 토너 500ml,3,스킨케어,스킨토너,NaN,4.43,85540,36000.0,NaN,스킨케어,토너/토너패드/스킨
3,화해,코스노리,판테놀 베리어 토너 205ml (2개입),4,스킨케어,스킨토너,NaN,4.76,2542,21900.0,NaN,스킨케어,토너/토너패드/스킨
4,화해,이즈앤트리,초저분자 히아루론산 토너 기획(300ml 본품+200ml 리필+미스트펌프),5,스킨케어,스킨토너,NaN,4.71,7026,15900.0,NaN,스킨케어,토너/토너패드/스킨


In [31]:
# 중분류2 복제
hwahae_df['중분류3'] = hwahae_df['중분류2']
hwahae_df['중분류3'].head()

0    토너/토너패드/스킨
1    토너/토너패드/스킨
2    토너/토너패드/스킨
3    토너/토너패드/스킨
4    토너/토너패드/스킨
Name: 중분류3, dtype: object

In [33]:
# '중분류2 = 립메이크업 제품' 소분류 카테고리 고유값 확인
cond = hwahae_df['중분류2'] == '립메이크업'
hwahae_df.loc[cond, '중분류'].unique()

array(['립스틱', '립틴트', '립글로스', '립케어_립밤', '컬러립케어_립밤'], dtype=object)

In [35]:
# 립메이크업- 립글로스/립밤/립스틱/립라이너/립틴트 

lipbalm_list = ['립밤']
lipbalm = '|'.join(lipbalm_list)
cond_lipbalm = hwahae_df['중분류'].str.contains(lipbalm)

for i in range(len(hwahae_df)):
  if cond_lipbalm[i] == True:
   hwahae_df.loc[i, '중분류3'] = '립밤'
  
  else:
    if hwahae_df['중분류'][i] == '립글로스':
      hwahae_df.loc[i, '중분류3'] = '립글로스'
      
    elif hwahae_df['중분류'][i] == '립스틱':
      hwahae_df.loc[i, '중분류3'] = '립스틱'
      
    elif hwahae_df['중분류'][i] == '립라이너':
      hwahae_df.loc[i, '중분류3'] = '립라이너'

    elif hwahae_df['중분류'][i] == '립틴트':
      hwahae_df.loc[i, '중분류3'] = '립틴트'


# 중분류3 입력값 확인
cond = hwahae_df['중분류2'] == '립메이크업'
print(hwahae_df.loc[cond, '중분류3'].unique())
print(hwahae_df.loc[cond, '중분류3'].nunique())
hwahae_df.loc[cond, ['중분류', '중분류3']].head()

['립스틱' '립틴트' '립글로스' '립밤']
4


,중분류,중분류3
4323,립스틱,립스틱
4324,립스틱,립스틱
4325,립스틱,립스틱
4326,립스틱,립스틱
4327,립스틱,립스틱


## 데이터셋 정리

In [36]:
# 기존 카테고리 지우기
hwahae_df.drop(columns={'중분류', '소분류'}, axis=1, inplace=True)
hwahae_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   평점      4781 non-null   float64
 6   후기 수    4781 non-null   int64  
 7   가격      4686 non-null   float64
 8   URL     0 non-null      float64
 9   중분류1    4781 non-null   object 
 10  중분류2    4781 non-null   object 
 11  중분류3    4781 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 448.3+ KB


In [37]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['중분류1', '중분류2', '중분류3']

before_a = hwahae_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[:5].to_list()
after_a = hwahae_df.drop(columns = {'중분류1', '중분류2', '중분류3'}).columns[5:].to_list()

reorder = before_a + a + after_a
hwahae_df = hwahae_df[reorder]

hwahae_df.columns

Index(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점',
       '후기 수', '가격', 'URL'],
      dtype='object')

In [38]:
# 최종 데이터셋 확인
print(hwahae_df.info())
hwahae_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4781 entries, 0 to 4780
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     4781 non-null   object 
 1   브랜드     4781 non-null   object 
 2   제품명     4781 non-null   object 
 3   중분류_순위  4781 non-null   int64  
 4   대분류     4781 non-null   object 
 5   중분류1    4781 non-null   object 
 6   중분류2    4781 non-null   object 
 7   중분류3    4781 non-null   object 
 8   평점      4781 non-null   float64
 9   후기 수    4781 non-null   int64  
 10  가격      4686 non-null   float64
 11  URL     0 non-null      float64
dtypes: float64(3), int64(2), object(7)
memory usage: 448.3+ KB
None


,플랫폼,브랜드,제품명,중분류_순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격,URL
0,화해,에스네이처,아쿠아 오아시스 토너 300ml 더블 기획(토너+토너+화장솜),1,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.73,17212,18900.0,NaN
1,화해,토리든,다이브인 저분자 히알루론산 토너 300ml 더블기획(+클렌징폼 30 ml),2,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.69,25849,19900.0,NaN
2,화해,라운드랩,1025 독도 토너 200ml + 독도 토너 500ml,3,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.43,85540,36000.0,NaN
3,화해,코스노리,판테놀 베리어 토너 205ml (2개입),4,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.76,2542,21900.0,NaN
4,화해,이즈앤트리,초저분자 히아루론산 토너 기획(300ml 본품+200ml 리필+미스트펌프),5,스킨케어,스킨케어,토너/토너패드/스킨,토너/토너패드/스킨,4.71,7026,15900.0,NaN


In [ ]:
# 최종 데이터셋 저장
# hwahae_df.to_csv('hwahae_preprocessing.csv', index=False, encoding="utf-8-sig")